In [6]:
import numpy as np
import networkx as nx
from itertools import combinations
import time  # süre ölçmek için eklendi

def read_tsp_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    n = int(lines[0].strip())
    cities = []
    for line in lines[1:]:
        parts = line.strip().split()
        cities.append((float(parts[0]), float(parts[1])))
    return n, cities

def distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

def compute_distance_matrix(cities):
    n = len(cities)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i][j] = distance(cities[i], cities[j])
    return dist_matrix

def christofides_tsp(cities):
    n = len(cities)
    dist_matrix = compute_distance_matrix(cities)
    G = nx.Graph()
    
    for i in range(n):
        for j in range(i+1, n):
            G.add_edge(i, j, weight=dist_matrix[i][j])
    
    mst = nx.minimum_spanning_tree(G)
    odd_vertices = [v for v in mst.nodes() if mst.degree(v) % 2 != 0]
    subgraph = G.subgraph(odd_vertices)

    try:
        matching = nx.algorithms.matching.min_weight_matching(subgraph, maxcardinality=True)
    except TypeError:
        matching = nx.algorithms.matching.max_weight_matching(subgraph, maxcardinality=True)
        matching = nx.algorithms.matching.max_weight_matching(subgraph, maxcardinality=True, 
                                                              weight=lambda u, v: -dist_matrix[u][v])
    
    multigraph = nx.MultiGraph(mst)
    for u, v in matching:
        multigraph.add_edge(u, v, weight=dist_matrix[u][v])
    
    try:
        eulerian_circuit = list(nx.eulerian_circuit(multigraph))
    except nx.NetworkXError:
        return list(mst.nodes()) + [list(mst.nodes())[0]], sum(mst.edges[e]['weight'] for e in mst.edges)
    
    path = []
    visited = set()
    for u, v in eulerian_circuit:
        if u not in visited:
            path.append(u)
            visited.add(u)
    path.append(path[0])
    
    total_cost = sum(dist_matrix[path[i]][path[i+1]] for i in range(len(path)-1))
    return path, total_cost

def two_opt_swap(path, dist_matrix):
    n = len(path)
    best_path = path
    best_cost = sum(dist_matrix[path[i]][path[i+1]] for i in range(n-1))
    improved = True
    
    while improved:
        improved = False
        for i in range(1, n-2):
            for j in range(i+1, n-1):
                if j == i:
                    continue
                new_path = path[:i] + path[i:j+1][::-1] + path[j+1:]
                new_cost = sum(dist_matrix[new_path[k]][new_path[k+1]] for k in range(n-1))
                if new_cost < best_cost:
                    best_path = new_path
                    best_cost = new_cost
                    improved = True
        path = best_path
    return best_path, best_cost

def christofides_with_2opt(cities):
    path, cost = christofides_tsp(cities)
    dist_matrix = compute_distance_matrix(cities)
    improved_path, improved_cost = two_opt_swap(path, dist_matrix)
    return improved_path, improved_cost

# Ana program
if __name__ == "__main__":
    start_time = time.time()

    n, cities = read_tsp_file('tsp_150.txt')
    path, cost = christofides_with_2opt(cities)

    end_time = time.time()
    elapsed_time = end_time - start_time

    print("Optimized Path:", path)
    print("Total Cost:", cost)
    print(f"Çalışma Süresi: {elapsed_time:.2f} saniye")


Optimized Path: [0, 144, 35, 45, 55, 63, 95, 46, 13, 115, 133, 42, 147, 77, 64, 134, 40, 78, 67, 88, 110, 131, 148, 143, 60, 24, 107, 119, 5, 21, 68, 20, 11, 74, 126, 85, 8, 122, 27, 52, 118, 137, 125, 51, 70, 59, 90, 44, 111, 140, 29, 93, 16, 84, 65, 23, 17, 123, 92, 103, 66, 56, 135, 100, 83, 2, 6, 81, 106, 41, 98, 120, 49, 61, 127, 75, 9, 114, 86, 101, 89, 87, 3, 34, 94, 138, 116, 129, 141, 121, 105, 71, 139, 54, 31, 39, 1, 4, 28, 73, 80, 128, 76, 109, 142, 104, 97, 26, 15, 47, 18, 19, 10, 57, 124, 7, 136, 62, 58, 149, 96, 36, 82, 30, 91, 99, 14, 108, 132, 12, 72, 22, 33, 69, 38, 32, 37, 50, 146, 48, 145, 112, 113, 102, 25, 130, 43, 79, 117, 53, 0]
Total Cost: 27899.072868609877
Çalışma Süresi: 37.71 saniye
